This script analyze the output of O3 box model with source apportionment technique. The goal of this script is to find the O3 contribution from different source of VOCs from the box model for Redlands, CA

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import join, isfile